In [1]:
import os

# Define the correct path to your dataset folder
dataset_path = '/content/drive/My Drive/Colab Notebooks/Monkey Pox/Monkey Pox Dataset'

# Change the current working directory to the dataset path
os.chdir(dataset_path)

# List the files and folders in the dataset directory to confirm
os.listdir(dataset_path)


['Monkey Pox Dataset', 'Other Dataset', 'Normal Dataset']

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np

In [3]:
# Paths to the datasets
monkey_pox_path = '/content/drive/My Drive/Colab Notebooks/Monkey Pox/Monkey Pox Dataset'
other_poxes_path = '/content/drive/My Drive/Colab Notebooks/Monkey Pox/Other Dataset'
healthy_skin_path = '/content/drive/My Drive/Colab Notebooks/Monkey Pox/Normal Dataset'

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Training and validation data generators
train_gen = datagen.flow_from_directory(
    '/content/drive/My Drive/Colab Notebooks/Monkey Pox/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    '/content/drive/My Drive/Colab Notebooks/Monkey Pox/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Verify the shapes of the data
batch_x, batch_y = next(train_gen)
print("Batch x shape:", batch_x.shape)  # Should be (batch_size, height, width, channels)
print("Batch y shape:", batch_y.shape)  # Should be (batch_size, num_classes)

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes: Monkey Pox, Other Poxes, Healthy Skin
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_gen, epochs=10, validation_data=val_gen)


Found 2683 images belonging to 1 classes.
Found 670 images belonging to 1 classes.
Batch x shape: (32, 128, 128, 3)
Batch y shape: (32, 1)
Epoch 1/10
84/84 [==============================] - 206s 2s/step - loss: 4235585.5000 - accuracy: 0.4428 - val_loss: 7822405.0000 - val_accuracy: 1.0000
Epoch 2/10
84/84 [==============================] - 14s 172ms/step - loss: 473102688.0000 - accuracy: 0.4007 - val_loss: 301988832.0000 - val_accuracy: 1.0000
Epoch 3/10
84/84 [==============================] - 15s 173ms/step - loss: 5824414208.0000 - accuracy: 0.3854 - val_loss: 2222347776.0000 - val_accuracy: 1.0000
Epoch 4/10
84/84 [==============================] - 15s 176ms/step - loss: 29879736320.0000 - accuracy: 0.3746 - val_loss: 7297614848.0000 - val_accuracy: 1.0000
Epoch 5/10
84/84 [==============================] - 15s 174ms/step - loss: 96885571584.0000 - accuracy: 0.3671 - val_loss: 14923812864.0000 - val_accuracy: 1.0000
Epoch 6/10
84/84 [==============================] - 14s 169ms/s

In [5]:
# Custom probability calculation function
def calculate_probability(image):
    # Get the model prediction
    prediction = model.predict(image)
    X = prediction[0][0]  # Probability of monkeypox

    # Symptom categories based on the remaining probability
    Y = (1 - X) / 2
    Z = (1 - X) / 3
    W = (1 - X) / 6

    # Final probability of monkeypox infection
    final_probability = X + Y + Z + W
    return final_probability

# Example usage
# Assuming `image` is a preprocessed image ready for prediction
# final_probability = calculate_probability(image)
# print("Probability of Monkey Pox Infection:", final_probability)